In [4]:
# install dependancies
# !pip install langchain openai pypdf tiktoken chromadb
# !pip freeze > requirements.txt 

In [5]:
# !curl -o mm_htn_guidelines.pdf "https://pstat-live-media.s3.amazonaws.com/pdf_cache/policy/8093105/f867219b-a6bd-40a1-a8ec-ebfdbb3d8adc/Ambulatory%20Adult%20Hypertension%20Guideline.pdf"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  264k  100  264k    0     0   585k      0 --:--:-- --:--:-- --:--:--  584k


In [2]:
FILE_PATH = "./mm_htn_guidelines.pdf"

In [3]:
from dotenv import load_dotenv, find_dotenv
# import os

if load_dotenv(find_dotenv()):
    print("Environment variables loaded successfully!") 
    # print(os.getenv('OPENAI_API_KEY'))
else:
    print("Could not load environment variables.") 

Environment variables loaded successfully!


In [17]:
# document Loader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

pdf_loader = PyPDFLoader(FILE_PATH)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

pdf_data = pdf_loader.load() #load_and_split(text_splitter=text_splitter)  # can use this instead of .load() and then no need for below line
documents = text_splitter.split_documents(pdf_data)

In [34]:
documents

[Document(page_content='COPYStatus Activ e PolicyStat ID 8093105 \nOrigination 02/1997 \nEffectiv e 08/2021 \nLast Re vised 08/2021 Area Ambulat ory \nPatient Car e \nApplicability UMHS-Clinical \nPublic \nReferences Guideline \nAmbulat ory Adult Hyper tension Guideline \nPatient population: \nObjectiv es: \nKey aspects and r ecommendations: \n• Diagnosis \n◦ Although a single, car efully tak en blood pr essur e (BP) r eading ma y predict futur e \ncardiovascular risk, for clinical purposes this risk is better identified b y taking the \nmean BP le vel from r ecor dings o ver se veral visits. \n◦ Home and ambulat ory blood pr essur e monit oring helps impr ove BP contr ol, and \nidentifies " white coat" and " mask ed" hyper tension [IIA]*. \n◦ If home BP monit oring is used, car eful calibr ation of the BP monit or and thor ough \npatient education ar e essential. \n◦ Individuals with mean BP > 135/80 should be scr eened for diabetes [IB]. \n• Treatment \n◦ For patients without diabete

In [37]:
len(documents)

35

In [19]:
# from langchain.prompts.prompt import PromptTemplate
# prompt_template = """You are a clinical practice guideline expert. A healtcare professional will ask you a question pertaining to medical guidelines, and you will provide a helpful answer. Please do not recommended speaking to a healtcare professional as part of your answer. If you don't know the answer, just say that you don't know. Use the following context from the medical guidelines to answer the question:
# {context}
# Question: {question}
# """
# QA_PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [38]:
# create embedding
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [39]:
from langchain.vectorstores import FAISS
faiss_db = FAISS.from_documents(documents, embeddings)

In [40]:
from langchain.chains.question_answering import load_qa_chain
query = "What is the 90-day cost for metoprolol?"
faiss_db.similarity_search(query)

[Document(page_content='COPYpropranolol Inder al LA   40 mg \nBID 80 mg BID $4-5 $150-300 \nInder al XL  60 mg daily 80 mg \ndaily 120 mg \ndaily $27-54 $178-356 \nlabetalol Trandate/\nNormodyne  100 mg BID 200 mg \nBID 300 mg \nBID $24-41 $40-69 \nnadolol Corgard  40 mg daily 80 mg \ndaily 160 mg \ndaily $96-153 $127-350 \nmetoprolol tar trate Lopr essor   50 mg \nBID 100 mg \nBID $11-15 $134-268 \nmetoprolol \nsuccinate Toprol XL   100 mg \ndaily 200 mg \ndaily $27-45 $58-93 \nByst olic  2.5 mg daily 10-20 \nmg \ndaily 40 mg \ndaily  $78-159 \nnebiv olol Coreg  3.125 BID 12.5-25 \nmg BID 25 BID $6-7 $90-160 \ncarvedilol Coreg CR 10mg \ndaily 20 mg daily 40 mg \ndaily 80 mg \ndaily n/a $158 all \nDirect V asodilat ors \nHydralazine generic  25 mg TID 50 mg \nTID 100 mg \nTID $10-25 n/a \nmino xidil generic   5mg \nBID 20mg BID $11-28 n/a \nCentr ally-acting alpha-2 agonists \nclonidine Catapr es   0.1 TID 0.3 TID $8 all $180-275 \nMethyldopa generic  25 0mg TID 500 mg \nTID 1000 mg \n

In [41]:
from langchain.vectorstores import Chroma

chroma_db = Chroma.from_documents(
    documents=documents, 
    embedding=embeddings, 
    persist_directory="db"
)
# call persist() to ensure the embeddings are written to disk. This isn't necessary in a script - the 
# database will be automatically persisted when the client object is destroyed.
chroma_db.persist()

In [6]:
# !pip install faiss-cpu
# !pip freeze > requirements.txt

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
def qa_doc(query, docsearch):
    chain = load_qa_chain(OpenAI(), chain_type="stuff")
    docs = docsearch.similarity_search(query)
    chain.run(input_documents=docs, question=query)
    print("chain:", chain)
    print("docs", docs)

In [15]:
query = "what is the cheapest bp drug avail?"
qa_doc(query, docsearch)

chain: memory=None callbacks=None callback_manager=None verbose=False tags=None metadata=None input_key='input_documents' output_key='output_text' llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=OpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key='sk-PHjzmzeqoEHmMYv5w2BpT3Bl

In [16]:
# initialize chat
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=vectordb.as_retriever(), 
    memory=memory,
)

ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to mak

In [16]:
# chat
chat_history = []
def qa_doc(query)
    # user_input = input("> ")
    ai_response = conversation({
        "question": query,
        "chat_history": chat_history
    })
    print(ai_response['answer'])

Based on the provided information, the treatment choice for pregnant women with a blood pressure of 150/110 would typically involve the use of antihypertensive medication. The specific medication would depend on the individual's medical history and any other concurrent conditions. However, it is important to note that the management of hypertension during pregnancy should be done by a physician experienced in managing these conditions. They will consider the fetal well-being and choose the most appropriate medication. It is recommended that women considering pregnancy and who require treatment for hypertension should be on antihypertensive medication ideally three to six months prior to conception. ACE inhibitors, ARBs, and renin inhibitors are contraindicated in pregnancy.
The usual dose range for enalapril is 2.5 mg daily to 10 mg daily. The cost for a 30-day supply of enalapril ranges from $7 to $11.
Before starting treatment for hypertension, the following tests are recommended:

1

In [18]:
query1 = "what is the target bp for a patient with diabetes"
qa_doc(query1)

[]
The target blood pressure for a patient with diabetes is a systolic blood pressure of less than 140 mmHg and a diastolic blood pressure of less than 90 mmHg.
